In [ ]:
# Import packages
import torch
from datetime import datetime

In [1]:
# Training function

def train_model(model, device, criterion, optimizer, train_loader, valid_loader, n_epochs=5, early_stop_patience=2):
    
    model.train()
    the_last_loss = 100
    trigger_times = 0
    train_losses_per_epoch, valid_losses_per_epoch = [], []
    
    # modify this later and find a nicer solution
    for i in range(1, 10000):
        if len(train_loader) / i <= 20:
            print_every = i
            break
        
    
    for epoch in range(n_epochs):  # loop over the dataset multiple times
        
        running_loss = 0.0
        running_acc = 0.0
        total = 0
        correct = 0
        
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels, inds = data
            inputs = inputs.float()
            labels = torch.clone(labels.long())
            
            # sending input to GPU/CPU
            inputs, labels = inputs.to(device), labels.to(device)
            
            
            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, predicted = outputs.max(1)
            total += labels.numel()
            correct += predicted.eq(labels).sum().item()
          
            # Statistics
            running_loss += loss.item()
            train_acc = correct/total
            running_acc += train_acc
            
            
            if i % print_every == print_every-1 or i+1 == len(train_loader):
                now = datetime.now()
                current_time = now.strftime("%m/%d/%Y, %H:%M:%S")
                print('Time: {}. . [{}/{}, {}/{}] train_loss: {:.6}, train_acc: {:.3}'.format(current_time, epoch+1, n_epochs, i+1, len(train_loader), running_loss/(i+1), running_acc/(i+1)))
    
        train_losses_per_epoch.append(running_loss/len(train_loader))
        current_loss = validate_model(model, device, valid_loader, criterion)
        print('The Current Validation Loss:', current_loss)
            
        if current_loss > the_last_loss:
            trigger_times += 1
            print('Trigger Times:', trigger_times)
    
            if trigger_times >= early_stop_patience:
                print('Early Stopping!\nStart the test process.')
                return model, train_losses_per_epoch, valid_losses_per_epoch
                # this needs to be modified to start the testing process -> return model if turn into a function
    
        else:
            print('Trigger Times: 0')
            trigger_times = 0
    
        the_last_loss = current_loss
        valid_losses_per_epoch.append(current_loss)
    
    print('Finished Training')        
    return model, train_losses_per_epoch, valid_losses_per_epoch

In [2]:
def validate_model(model, device, valid_loader, loss_function):

     model.eval()
     loss_total = 0

     # Test validation data
     with torch.no_grad():
         for data in valid_loader:
             inputs, labels, inds = data
             inputs = inputs.float()
             labels = torch.clone(labels.long())
             
             #Sending input to GPU/CPU
             inputs, labels = inputs.to(device),  labels.to(device)
             

             output = model(inputs)
             loss = loss_function(output, labels)
             loss_total += loss.item()

     return loss_total / len(valid_loader)  

    
# Test function

def test_model(device, model, test_loader):
    
    model.eval()
    total = 0
    correct = 0
    accuracy = 0
    labels_all = torch.tensor([])
    predicted_all = torch.tensor([])
    
    with torch.no_grad():
        for data in test_loader:
            inputs, labels, inds = data
            inputs = inputs.float()
            labels = torch.clone(labels.long())
            
            # sending input to GPU/CPU
            inputs, labels = inputs.to(device), labels.to(device)
            
            output = model(inputs)
            _, predicted = torch.max(output.data, 1)

            total += labels.numel()
            correct += (predicted == labels).sum().item()
            
            
            ps = torch.exp(output)
            top_p, top_class = ps.topk(1, dim=1)
            equals =  top_class == labels.view(*top_class.shape)
            accuracy +=   torch.mean(equals.type(torch.FloatTensor)).item()
            predicted_all = torch.cat((predicted_all, predicted) ,dim=0)
            labels_all = torch.cat((labels_all, labels) ,dim=0)
            

    print('Accuracy: {:.2}'.format(correct/total))
    print('Accuracy2: {:.2}'.format(accuracy/len(test_loader)))
    return labels_all, predicted_all
          